In [1]:
%load_ext sql
import sqlite3
import pandas as pd

In [2]:
%sql sqlite:///CourseData.db
conn = sqlite3.connect('CourseData.db')

SQL DISTINCT DATABASE PROJECT
======
# 1. Design ERD from Source Data and Create Tables
![ERD](Database_ERD.png)

## A. Instructors Table

In [3]:
%%sql 
-- Creates instructors table
DROP TABLE IF EXISTS INSTRUCTORS;
CREATE TABLE INSTRUCTORS(
    INSTRUCTOR_ID INTEGER PRIMARY KEY,
    INSTRUCTOR_NAME TEXT NOT NULL);

 * sqlite:///CourseData.db
Done.
Done.


[]

## B. Terms Table

In [4]:
%%sql
-- Creates Terms table
DROP TABLE IF EXISTS TERMS;
CREATE TABLE TERMS(
    TERM_ID INTEGER PRIMARY KEY,
    SEMESTER TEXT NOT NULL,
    YEAR INTEGER NOT NULL,
    CRN INTEGER);

 * sqlite:///CourseData.db
Done.
Done.


[]

## C. Programs Table

In [5]:
%%sql
-- Creates programs table
DROP TABLE IF EXISTS PROGRAMS;
CREATE TABLE PROGRAMS(
    PROGRAM_ID INTEGER PRIMARY KEY,
    PROGRAM_CODE TEXT NOT NULL,
    PROGRAM_NAME TEXT NOT NULL);

 * sqlite:///CourseData.db
Done.
Done.


[]

## D. Courses Table

In [6]:
%%sql
-- Creates courses table
DROP TABLE IF EXISTS COURSES;
CREATE TABLE COURSES(
    COURSE_ID INTEGER PRIMARY KEY,
    CATALOG_ID TEXT,
    YEAR TEXT NOT NULL,
    TITLE TEXT NOT NULL,
    PREREQ TEXT,
    COREQ TEXT,
    FEES TEXT,
    ATTRIBUTES TEXT,
    DESCRIPTION TEXT,
    PROGRAM_ID INTEGER NOT NULL,
    FOREIGN KEY (PROGRAM_ID) 
        REFERENCES PROGRAM (PROGRAM_ID)
        ON UPDATE CASCADE
);

 * sqlite:///CourseData.db
Done.
Done.


[]

## E. Course Offerings Table

In [7]:
%%sql
-- Creates course offerings table
DROP TABLE IF EXISTS COURSE_OFFERINGS;
CREATE TABLE COURSE_OFFERINGS(
    COURSE_OFFERING_ID INTEGER PRIMARY KEY,
    CRN INTEGER NOT NULL,
    SECTION INTEGER NOT NULL,
    CREDITS INTEGER NOT NULL,
    CAPACITY INTEGER NOT NULL,
    ACTUAL INTEGER NOT NULL,
    REMAINING INTEGER NOT NULL,
    TERM TEXT NOT NULL,
    MEETINGS TEXT,
    TIMECODES TEXT,
    CATALOG_ID INTEGER NOT NULL,
    PRIMARY_INSTRUCTOR_ID INTEGER,
    FOREIGN KEY (CATALOG_ID)
        REFERENCES COURSES(CATALOG_ID)
        ON UPDATE CASCADE,
    FOREIGN KEY (PRIMARY_INSTRUCTOR_ID)
        REFERENCES INSTRUCTOR(INSTRUCTOR_ID)
        ON UPDATE CASCADE
);

 * sqlite:///CourseData.db
Done.
Done.


[]

## F. Locations Table

In [8]:
%%sql
-- Create locations table
DROP TABLE IF EXISTS LOCATIONS;
CREATE TABLE LOCATIONS(
    LOCATION_ID INTEGER PRIMARY KEY,
    ROOM_ID TEXT);

 * sqlite:///CourseData.db
Done.
Done.


[]

## G. Course Meetings Table

In [9]:
%%sql 
-- Create course meetings table 
DROP TABLE IF EXISTS COURSE_MEETINGS;
CREATE TABLE COURSE_MEETINGS(
    MEETING_ID INTEGER PRIMARY KEY,
    TERM TEXT NOT NULL,
    LOCATION TEXT NOT NULL,
    DAY TEXT NOT NULL,
    START TEXT NOT NULL,
    END TEXT NOT NULL,
    CRN INTEGER NOT NULL,
    COURSE_OFFERING_ID INTEGER NOT NULL,
    LOCATION_ID INTEGER,
    FOREIGN KEY (COURSE_OFFERING_ID)
        REFERENCES COURSE_OFFERING (COURSE_OFFERING_ID)
        ON UPDATE CASCADE,
    FOREIGN KEY (LOCATION_ID)
        REFERENCES LOCATION (LOCATION_ID)
        ON UPDATE CASCADE
);

 * sqlite:///CourseData.db
Done.
Done.


[]

## H. Term/Catalog Year Table

In [10]:
%%sql
-- Creates a table for the catalog year, which links two semesters together 
DROP TABLE IF EXISTS TERM_CATALOGYEAR;
CREATE TABLE TERM_CATALOGYEAR(
    TERM TEXT,
    CATALOG_YEAR TEXT);

 * sqlite:///CourseData.db
Done.
Done.


[]

*Had to manually insert data into TERM_CATALOGYEAR table*

In [11]:
%%sql
-- Manually insert the values for catlog year into the table
INSERT INTO TERM_CATALOGYEAR (TERM, CATALOG_YEAR) VALUES 
('Fall2017','2017_2018'),('Winter2018', '2017_2018'),('Spring2018','2017_2018'),('Summer2018','2017_2018'),
('Fall2018','2018_2019'),('Winter2019','2018_2019'),('Spring2019','2018_2019');

 * sqlite:///CourseData.db
Done.


[]

# 2. Import Data from the Source Data into our database tables

## A. Utilized python to upload source data files into temporary tables

In [12]:
# Import catalog data
catalog_year = ['2017_2018', '2018_2019']

for year in catalog_year:
    filepath = 'SourceData/Catalogs/CourseCatalog'+year+'.csv'
    catalog_data = pd.read_csv(filepath)
    catalog_data['year'] = year
    catalog_data.to_sql('IMPORT_CAT_COURSES',conn, if_exists ='append', index=False)

In [13]:
# Import Course meetings data and course offerings data
terms = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018',
         'Spring2015','Spring2016','Spring2017','Spring2018','Spring2019',
         'SpringBreak2017','Summer2015','Summer2016','Summer2017','Summer2018',
         'Winter2015','Winter2016','Winter2017','Winter2018']
for term in terms:
    filepath = 'SourceData/'+term+'/courses.csv'
    term_data = pd.read_csv(filepath)
    term_data.to_sql('IMPORT_COURSE_DATA',conn,if_exists='append',index=False)
    
    filepath = 'SourceData/'+term+'/course_meetings.csv'
    term_data = pd.read_csv(filepath)
    term_data.to_sql('IMPORT_COURSEMEETINGS_DATA',conn, if_exists='append',index=False)

### Checked the data with select queries to ensure all data was uploaded

In [14]:
%%sql
SELECT
    (SELECT COUNT(*) FROM IMPORT_COURSE_DATA) as 'RAW_COUNT',
    (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_DATA)) as 'DISTINCT_COUNT';

 * sqlite:///CourseData.db
Done.


RAW_COUNT,DISTINCT_COUNT
15937,15937


In [15]:
%%sql
SELECT 
    (SELECT COUNT(*) FROM IMPORT_CAT_COURSES) as 'RAW_COUNT',
    (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM IMPORT_CAT_COURSES)) as 'DISTINCT_COUNT';

 * sqlite:///CourseData.db
Done.


RAW_COUNT,DISTINCT_COUNT
4440,4440


In [16]:
%%sql
SELECT
    (SELECT COUNT(*) FROM IMPORT_COURSEMEETINGS_DATA) as 'RAW_COUNT',
    (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSEMEETINGS_DATA)) as 'DISTINCT_COUNT';

 * sqlite:///CourseData.db
Done.


RAW_COUNT,DISTINCT_COUNT
284907,284847


*Difference of 60 because of a deficiency in the course meetings data*

# 3. Uplaod data into the CourseData.db tables

## A. Import data into INSTRUCTORS table

In [17]:
%%sql
DELETE FROM INSTRUCTORS;

INSERT INTO INSTRUCTORS (INSTRUCTOR_NAME)
SELECT DISTINCT primary_instructor
FROM IMPORT_COURSE_DATA 
WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE '%/%';

 * sqlite:///CourseData.db
0 rows affected.
1095 rows affected.


[]

In [18]:
%%sql
-- Ran select query to verify data loaded properly
SELECT *
FROM INSTRUCTORS
LIMIT 10;

 * sqlite:///CourseData.db
Done.


INSTRUCTOR_ID,INSTRUCTOR_NAME
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos
6,Scott M Brenner
7,Kevin C. Cassidy
8,Bruce Bradford
9,Milo W. Peck
10,Stephen E. Yost


## B. Import data into PROGRAMS table

In [19]:
%%sql
DELETE FROM PROGRAMS;

INSERT INTO PROGRAMS (PROGRAM_CODE, PROGRAM_NAME)
SELECT DISTINCT program_code, program_name
FROM IMPORT_CAT_COURSES
ORDER BY PROGRAM_CODE;

 * sqlite:///CourseData.db
0 rows affected.
83 rows affected.


[]

In [20]:
%%sql
-- Ran select query to verify data loaded properly
SELECT *
FROM PROGRAMS
LIMIT 10;

 * sqlite:///CourseData.db
Done.


PROGRAM_ID,PROGRAM_CODE,PROGRAM_NAME
1,AC,Accounting
2,AE,Applied Ethics
3,AH,Art History
4,AN,Asian Studies
5,AR,Arabic
6,AS,American Studies
7,AY,Anthropology
8,BB,Business
9,BEN,Bioengineering
10,BI,Biology


## C. Import data into LOCATIONS table

In [21]:
%%sql
DELETE FROM LOCATIONS;

INSERT INTO LOCATIONS (ROOM_ID)
SELECT DISTINCT location
FROM IMPORT_COURSEMEETINGS_DATA
ORDER BY location;

 * sqlite:///CourseData.db
0 rows affected.
207 rows affected.


[]

In [22]:
%%sql
-- Ran select query to verify data loaded properly
SELECT *
FROM LOCATIONS
LIMIT 10;

 * sqlite:///CourseData.db
Done.


LOCATION_ID,ROOM_ID
1,BCC 200
2,BD
3,BH
4,BH BY ARR
5,BLM 112
6,BLM LL105
7,BNW 124
8,BNW 127
9,BNW 128
10,BNW 129B


## D. Import data into COURSE_OFFERINGS table

In [23]:
%%sql
DELETE FROM COURSE_OFFERINGS;

INSERT INTO COURSE_OFFERINGS (CRN, SECTION, CREDITS, CAPACITY, ACTUAL, REMAINING, CATALOG_ID, TERM, MEETINGS, TIMECODES, 
                             PRIMARY_INSTRUCTOR_ID)
SELECT DISTINCT crn, section, credits, cap, act, rem, catalog_id, term, meetings, timecodes, instructor_id
FROM IMPORT_COURSE_DATA
    LEFT JOIN INSTRUCTORS ON (IMPORT_COURSE_DATA.primary_instructor = INSTRUCTORS.INSTRUCTOR_NAME);

 * sqlite:///CourseData.db
0 rows affected.
15937 rows affected.


[]

In [24]:
%%sql
-- Ran select query to verify data loaded properly
SELECT *
FROM COURSE_OFFERINGS
LIMIT 5;

 * sqlite:///CourseData.db
Done.


COURSE_OFFERING_ID,CRN,SECTION,CREDITS,CAPACITY,ACTUAL,REMAINING,TERM,MEETINGS,TIMECODES,CATALOG_ID,PRIMARY_INSTRUCTOR_ID
1,70384,C01,3,0,31,-31,Fall2014,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],AC 0011,1
2,70385,C02,3,0,31,-31,Fall2014,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0930am-1045am 09/02-12/08 DSB 105'],AC 0011,1
3,70382,C03,3,0,31,-31,Fall2014,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],AC 0011,1
4,70291,C04,3,0,29,-29,Fall2014,"[{'days': 'MR', 'times': '1100am-1215pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1100am-1215pm 09/02-12/08 DSB 111'],AC 0011,2
5,70350,C05,3,0,30,-30,Fall2014,"[{'days': 'MR', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1230pm-0145pm 09/02-12/08 DSB 111'],AC 0011,2


## E. Import data into COURSES table

In [25]:
%%sql
DELETE FROM COURSES;

INSERT INTO COURSES (CATALOG_ID, YEAR, TITLE, PREREQ, COREQ, FEES, ATTRIBUTES, DESCRIPTION, PROGRAM_ID)
SELECT DISTINCT IMPORT_CAT_COURSES.catalog_id, year, course_title, prereqs, coreqs, fees, attributes, description, PROGRAMS.PROGRAM_ID
FROM IMPORT_CAT_COURSES
    LEFT JOIN COURSE_OFFERINGS USING (CATALOG_ID)
    JOIN PROGRAMS ON (PROGRAMS.PROGRAM_CODE = IMPORT_CAT_COURSES.program_code);

 * sqlite:///CourseData.db
0 rows affected.
4440 rows affected.


[]

In [26]:
%%sql
-- Ran select query to verify data loaded properly
SELECT *
FROM COURSES
LIMIT 5;

 * sqlite:///CourseData.db
Done.


COURSE_ID,CATALOG_ID,YEAR,TITLE,PREREQ,COREQ,FEES,ATTRIBUTES,DESCRIPTION,PROGRAM_ID
1,AN 0301,2017_2018,Independent Study,None,None,None,None,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.,4
2,AN 0310,2017_2018,Asian Studies Seminar,None,None,None,None,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only.",4
3,BU 0211,2017_2018,Legal Environment of Business,Junior standing.,None,None,None,"This course examines the broad philosophical as well as practical nature and function of the legal system, and introduces students to the legal and social responsibilities of business. The course includes an introduction to the legal system, the federal courts, Constitutional law, the United States Supreme Court, the civil process, and regulatory areas such as employment discrimination, protection of the environment, and corporate governance and securities markets.",12
4,BU 0220,2017_2018,Environmental Law and Policy,None,None,None,"EVME Environmental Studies Major Elective, EVPE Environmental Studies Elective, EVSS Environmental Studies: Social Science, MGEL Management: General Elective","This course surveys issues arising out of federal laws designed to protect the environment and manage resources. It considers in detail the role of the Environmental Protection Agency in the enforcement of environmental policies arising out of such laws as the National Environmental Policy Act, the Clean Water Act, and the Clear Air Act, among others. The course also considers the impact of Congress, political parties, bureaucracy, and interest groups in shaping environmental policy, giving special attention to the impact of environmental regulation on business and private property rights.",12
5,BU 0311,2017_2018,"The Law of Contracts, Sales, and Property",BU 0211.,None,None,None,"This course examines the components of common law contracts including the concepts of offer and acceptance, consideration, capacity and legality, assignment of rights and delegation of duties, as well as discharge of contracts. The course covers Articles 2 and 2A of the Uniform Commercial Code relating to leases, sales of goods, and warranties. The course also considers personal and real property, and bailments.",12


## F. Import data into COURSE_MEETINGS

In [27]:
%%sql
DELETE FROM COURSE_MEETINGS;

INSERT INTO COURSE_MEETINGS (TERM, LOCATION, DAY, START, END, CRN, COURSE_OFFERING_ID, LOCATION_ID)
SELECT term, location, day, start, end, CRN , COURSE_OFFERING_ID, LOCATIONS.LOCATION_ID
FROM IMPORT_COURSEMEETINGS_DATA
    LEFT JOIN LOCATIONS ON (IMPORT_COURSEMEETINGS_DATA.location = LOCATIONS.ROOM_ID)
    JOIN COURSE_OFFERINGS USING (CRN,TERM);

 * sqlite:///CourseData.db
0 rows affected.
284907 rows affected.


[]

In [28]:
%%sql
-- Ran select query to verify data loaded properly
SELECT *
FROM COURSE_MEETINGS
LIMIT 10;

 * sqlite:///CourseData.db
Done.


MEETING_ID,TERM,LOCATION,DAY,START,END,CRN,COURSE_OFFERING_ID,LOCATION_ID
1,Fall2014,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00,70384,1,99
2,Fall2014,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00,70384,1,99
3,Fall2014,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00,70384,1,99
4,Fall2014,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00,70384,1,99
5,Fall2014,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00,70384,1,99
6,Fall2014,DSB 105,F,2014-09-19T08:00:00,2014-09-19T09:15:00,70384,1,99
7,Fall2014,DSB 105,T,2014-09-23T08:00:00,2014-09-23T09:15:00,70384,1,99
8,Fall2014,DSB 105,F,2014-09-26T08:00:00,2014-09-26T09:15:00,70384,1,99
9,Fall2014,DSB 105,T,2014-09-30T08:00:00,2014-09-30T09:15:00,70384,1,99
10,Fall2014,DSB 105,F,2014-10-03T08:00:00,2014-10-03T09:15:00,70384,1,99


In [29]:
%%sql
-- Delete all data
DELETE FROM IMPORT_CAT_COURSES;
DELETE FROM IMPORT_COURSE_DATA;
DELETE FROM IMPORT_COURSEMEETINGS_DATA;

 * sqlite:///CourseData.db
Done.
15937 rows affected.
284907 rows affected.


[]

In [30]:
%%sql
-- Drop the tables
DROP TABLE IMPORT_CAT_COURSES;
DROP TABLE IMPORT_COURSE_DATA;
DROP TABLE IMPORT_COURSEMEETINGS_DATA;

 * sqlite:///CourseData.db
Done.
Done.
Done.


[]

In [31]:
%%sql
-- Rebuil file to save storage space
vacuum;

 * sqlite:///CourseData.db
Done.


[]